## test bleach correction

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pathlib import Path
from pystripe.core import imread_tif_raw_png, glob_re, log1p_jit, expm1_jit, correct_bleaching
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu


files_path = Path(r"/qnap/SmartSPIM_Data/2023_04_02/20230402_10_49_31_SM220712_01_LS_15x_800z/Ex_488_Em_525_MIP")
for file in glob_re(r"tiff?", files_path):
    img = imread_tif_raw_png(file)
    if img.min() < img.max():
        img_filtered = log1p_jit(img)
        threshold = threshold_otsu(img_filtered)
        print(threshold)
        if threshold < 3:
            continue
        
        img_filtered = expm1_jit(correct_bleaching(img_filtered, 0.0005, max_method=True))
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 10))
        axes[0].imshow(img, cmap='gray', vmin=0, vmax=2047)
        axes[0].set_title("original")
        axes[1].imshow(img_filtered, cmap='gray', vmin=0, vmax=2047)
        axes[1].set_title("bleach corrected")
        plt.tight_layout()
        plt.show()

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pystripe.core import *
import matplotlib.pyplot as plt
def plot_images(img_list: List[ndarray], img_labels: List[str], vmax: int):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 20))
    for idx, (im, label) in enumerate(zip(img_list, img_labels)):
        axes[idx].imshow(im, cmap='gray', vmin=0, vmax=vmax)
        axes[idx].set_title(label)
    plt.tight_layout()
    plt.show()

# from tsv.volume import TSVVolume, VExtent
# file = Path(r"/data/20230402_SM220712_01_LS_15x_800z_stitched/Ex_488_Em_525_xml_import_step_5.xml")
# tsv_volume = TSVVolume.load(file)
# x0, x1, y0, y1 = tsv_volume.volume.x0, tsv_volume.volume.x1, tsv_volume.volume.y0, tsv_volume.volume.y1
# idx = 3001
# img = tsv_volume.imread(VExtent(x0, x1, y0, y1, idx, idx + 1), "uint16")[0]
img = imread_tif_raw_png(Path(r"/data/20230402_SM220712_01_LS_15x_800z_stitched/Ex_488_Em_525_tif/img_003000.tif"))

In [ ]:
imgs = tuple(imread_tif_raw_png(Path(f"/data/20230402_SM220712_01_LS_15x_800z_stitched/Ex_488_Em_525_tif/img_{idx:06}.tif")) for idx in range(1000, 10000, 1000))
for img in imgs:
    img_destriped = process_img(
        img.copy(),
        sigma=(4000, 4000), bidirectional=False, bleach_correction_frequency=1/2000, bleach_correction_clip_max=511, exclude_dark_edges_set_them_to_zero=True, bleach_correction_y_slice_max=None,
        # lightsheet=True, percentile=.025, background_window_size=100, artifact_length=90,
        verbose=True
    )
    plot_images([img, img_destriped], ["original", "processed"], vmax=2047)

In [ ]:
def slice_non_zero_box(img_axis: ndarray, noise: int, filter_frequency: int = 1/1000) -> (int, int):
    return min_max_1d(nonzero(butter_lowpass_filter(img_axis, filter_frequency).astype(uint16) > noise)[0])


noise_percentile = 5
foreground_percentile = 100 - noise_percentile
# idx = 4000
imgs = tuple(imread_tif_raw_png(Path(f"/data/20230402_SM220712_01_LS_15x_800z_stitched/Ex_488_Em_525_tif/img_{idx:06}.tif")) for idx in range(1000, 10000, 1000))
maxes = tuple((np_max(img, axis=0), np_max(img, axis=1)) for img in imgs)
for idx, (img, (img_max_x, img_max_y)) in enumerate(zip(imgs, maxes), start=1):

    x_slice_min, x_slice_max = slice_non_zero_box(img_max_x, prctl(img_max_x, noise_percentile))
    y_slice_min, y_slice_max = slice_non_zero_box(img_max_y, prctl(img_max_y, noise_percentile))
    print(
        f"{idx}  min  max  end\n"
        f"x {x_slice_min} {x_slice_max} {img.shape[1]}\n"
        f"y {y_slice_min} {y_slice_max} {img.shape[0]}\n"
        f"work {(x_slice_max - x_slice_min) * (y_slice_max - y_slice_min) / (img.shape[0] * img.shape[1])*100:.1f}%\n"
    )
    tile_size = img.shape
    d_type = img.dtype
    img_sliced = img[y_slice_min:y_slice_max, x_slice_min:x_slice_max]
    img_zero = zeros(shape=tile_size, dtype=d_type)
    img_zero[y_slice_min:y_slice_max, x_slice_min:x_slice_max] = img_sliced
    plot_images([img, img_zero], ["original", "processed"], vmax=100)
